In [15]:
%load_ext watermark
%watermark -a Filippo_Valle -p pandas,numpy,graph_tool,cloudpickle -m -v -g

Author: Filippo_Valle

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.22.0

pandas     : 1.2.3
numpy      : 1.19.0
graph_tool : 2.37 (commit afba9459, )
cloudpickle: 1.6.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.11.0-16-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

Git hash: 5d27b88efae870eb8ba529aaac3fbde1772bb42e



In [ ]:
import graph_tool.all as gt
import pandas as pd
import numpy as np
import cloudpickle as pickle
import sys
sys.path.append("../trisbm")
from trisbm import trisbm

In [ ]:
with open("brca/trisbm/model.pkl", "rb") as file:
    model = pickle.load(file)

In [ ]:
#network_file = "mirdip/HUMAN.mirDIP_top90k.Translated.tsv"
network_file = "tarbase/HUMAN.TarBase.Translated.tsv"

In [ ]:
df_net = pd.read_csv("../regulatory/"+network_file, sep="\t", header=None)
df_net[0]=[s.replace("miR","mir") for s in df_net[0]]
df_conversion = pd.read_csv("miRNA.txt")
df_gene_conversion = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=gd_pub_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", sep="\t").dropna(how="any", axis=0)
df_net = df_net[df_net[0].isin(model.keywords)]
df_net = df_net[df_net[0].isin(df_conversion["miRBase ID"])]
df_net = df_net[df_net[1].isin(df_gene_conversion["Approved symbol"])]
df_gene_conversion = df_gene_conversion.set_index("Approved symbol")

In [ ]:
vertexmap = {}
for i,node in enumerate(np.concatenate((model.documents, model.words, model.keywords))):
    vertexmap[node]=i

In [ ]:
def get_regulatory_edge():
    for mirna, target in df_net.values:
        gene = df_gene_conversion.at[target, "Ensembl gene ID"]
        if (mirna in model.keywords) and (gene in model.words):
            #print(mirna, target, gene)
            yield (vertexmap[mirna], vertexmap[gene], 1.)
regulatory_edges = get_regulatory_edge()

In [ ]:
print(model.g)
model.g.save("tmp.xml.gz") #need to be stored to remove edges later

In [ ]:
model.g.add_edge_list(regulatory_edges, eprops=[model.g.ep["count"]])

In [ ]:
print(model.g)

In [ ]:
state = model.state.copy(g=model.g, bs=model.state.get_bs() + [np.zeros(1)] * 4, sampling = True)

S1 = state.entropy()

for i in range(100):
    state.multiflip_mcmc_sweep(niter=10, beta=np.inf)
    
print(model.get_mdl(), S1, state.entropy())

In [ ]:
model.state = state
model.g = gt.load_graph("tmp.xml.gz")

In [ ]:
import os

In [ ]:
# necessary to be compatible with older versions of trisbm
model.nbranches = 1 # not stored in older versions
model.keywords = [model.keywords] # new format
model.groups = {} # should not be cached

In [ ]:
os.chdir("brca")
os.system("rm -rf tarbase && mkdir -p tarbase")
os.chdir("tarbase")
os.system("mkdir -p trisbm")
os.chdir("trisbm")
model.save_data()
os.chdir("../../../")

In [ ]:
os.chdir("../../")